In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
from load_paus_cat import load_paus_cat
from jpasLAEs.utils import flux_to_mag
from paus_utils import w_central

field_name = 'W3'
path_to_cat = f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_{field_name}.csv'
cat = load_paus_cat(path_to_cat)

In [ ]:
cat['flx'][45]

In [ ]:
def expfit(x, a, b, c):
    return a * np.exp(b * x + c)

def linfit(x, a, b):
    return a * x + b

In [ ]:
NB_number_mask = (cat['NB_number'] > 39)
flx = cat['flx'][:, NB_number_mask]
err = cat['err'][:, NB_number_mask]
flx_relerr = err / flx

flx[~np.isfinite(flx)] = 1e-99

err_fit_params = []

for filter_i in range(46):
    # if filter_i == 45:
    #     err_fit_params.append([0., 0., 0.])
    #     continue
    mask = (flx[filter_i] > 1e-18) & (flx[filter_i] < 1e-15)
    p0 = [1., 1., 1.]
    p, var = curve_fit(expfit, np.log10(flx[filter_i, mask]),
                       flx_relerr[filter_i, mask], p0=p0)
    err_fit_params.append(p)

print(err_fit_params)

In [ ]:
filter_i = 45

fig, ax = plt.subplots(figsize=(6, 4))

ax.scatter(flx[filter_i], flx_relerr[filter_i], s=1)

flx_x = np.linspace(-19, -15, 1000)
[a, b, c] = err_fit_params[filter_i]
ax.plot(10**flx_x, expfit(flx_x, a, b, c), ls='--', c='k')
print(err_fit_params[filter_i])

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlim(1e-19, 1e-15)

plt.show()

In [ ]:
path_to_corr = '/home/alberto/almacen/PAUS_data/catalogs/error_distribution'
np.save(f'{path_to_corr}/fit_params_{field_name}.npy', np.array(err_fit_params))